In [1]:
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv(), override=True)

In [2]:
# loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file, )
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
        
    data = loader.load()
    return data

def split_data(data, llm, chunk_size=4096):
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                                   length_function=lambda x: llm.get_num_tokens(x),
                                                  )#separators=[" ", "\n", "\n\n"])

    chunks = text_splitter.split_documents(data)
    return chunks

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3")

In [3]:
data = load_document("msg_life-gb-2021-EN_final.txt")
chunks = split_data(data, llm=llm)

In [4]:
print(f"number of words in document: {sum([len(chunk.page_content.split()) for chunk in chunks])}\
\nnumber of chunks: {len(chunks)}\
\nnumber of words in 5th chunk: {len(chunks[5].page_content.split())}\
\nnumber of tokens in 5th chunk: {llm.get_num_tokens(chunks[5].page_content)}")

number of words in document: 30165
number of chunks: 12
number of words in 5th chunk: 3168
number of tokens in 5th chunk: 4034


In [19]:
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.prompts.prompt import PromptTemplate

SYSTEM_MESSAGE = """Suppose you are QuestionAnswerGPT, a large language model trained in detecting the most important
parts of a document and come up with a corresponding question and answer pair for these highly important sections in the
document, that would help new readers of the document.

Your task is to generate questions and corresponding answers from the given task that sum it up best and that
would allow us to build a multiple choice task. Additionally you have to explain your decision by attatching 
a number or score to the question - answer pair that tells us how important it is. The score is between 0 and 1.
```
{{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE",
    "score": "$YOUR_IMPORTANCE_SCORE",
}}
```
Everything between the ``` must be valid json. EXAMPLES:

{'question': 'What is the headquarters address of the msg life Group?',
 'answer': 'The headquarters of the msg life Group are at Humboldtstrasse 35, 70771 Leinfelden-Echterdingen, Germany.'},
 'score': 0.7
"""

HUMAN_MESSAGE = """Please generate questions, answers and importance scores in valid JSON format
of the following text: {text}"""

CHAT_PROMPT = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(SYSTEM_MESSAGE),
        HumanMessagePromptTemplate.from_template(HUMAN_MESSAGE),
    ])

In [23]:
from langchain import PromptTemplate
from langchain.chains import QAGenerationChain

chain_output = QAGenerationChain.from_llm(llm=llm, prompt=CHAT_PROMPT)

chain_output = qa_chain.run(chunks[5].page_content)

chain_output

JSONDecodeError: Extra data: line 6 column 1 (char 261)